In [150]:
import pandas as pd

In [151]:
pelis = pd.read_csv('pelis_combinado.csv')

In [152]:
pelis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1955 entries, 0 to 1954
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Nombre         1955 non-null   object
 1   Fecha          1955 non-null   object
 2   Tipo           1691 non-null   object
 3   Horario        1955 non-null   object
 4   FormatoImagen  1691 non-null   object
 5   FormatoIdioma  1691 non-null   object
 6   Cine           1955 non-null   object
dtypes: object(7)
memory usage: 107.0+ KB


In [153]:
pelis['Fecha'] = pd.to_datetime(pelis['Fecha'])
pelis.drop(columns=['Tipo'], inplace=True)

In [154]:
pelis.sort_values(by='Fecha', inplace=True)

pelis.set_index('Fecha', inplace=True)

In [155]:
pelis

,Nombre,Horario,FormatoImagen,FormatoIdioma,Cine
Fecha,,,,,
2025-04-23,MINECRAFT,18:30,2D,Español,Monumental
2025-04-23,PECADORES,21:00,2D,Español,Monumental
2025-04-23,MAZEL TOV,21:15,2D,Español,Monumental
2025-04-23,MAZEL TOV,19:00,2D,Español,Monumental
2025-04-23,BLANCANIEVES,19:40,2D,Español,Monumental
...,...,...,...,...,...
2025-05-07,UNTIL DAWN NOCHE DE TERROR,14:00,2D,Español,Cinemark Hoyts
2025-05-07,UNTIL DAWN NOCHE DE TERROR,16:50,2D,Español,Cinemark Hoyts
2025-05-07,UNTIL DAWN NOCHE DE TERROR,19:40,2D,Español,Cinemark Hoyts


---

In [156]:
print(pelis['Nombre'].unique())

['MINECRAFT' 'PECADORES' 'MAZEL TOV' 'BLANCANIEVES' 'Mazel Tov'
 'DROP AMENAZA ANONIMA' 'Pecadores' 'El Amateur: Operación Venganza'
 'Una película de Minecraft' 'Tierras perdidas'
 'EL AMATEUR: OPERACION VENGANZA' 'DROP: Amenaza anónima'
 'TIERRAS PERDIDAS' 'DROP: Amenaza Anónima' 'Blanca Nieves'
 'Rescate implacable' 'The Chosen: La última cena' 'Looney Tunes'
 'The chosen: la última cena' 'Looney Tunes: El Día que la Tierra Explotó'
 'Until dawn: noche de terror' 'Flow' 'Cónclave'
 'La venganza de los Sith 20 Aniversario' 'Puente en llamas'
 'El Contador 2' 'Until dawn – Noche de Terror'
 'Pink Floyd at Pompeii – MCMLXXII' 'El contador 2' 'EL REY DE REYES'
 'El día que la Tierra explotó: Una película de Looney Tunes' 'Until Dawn'
 'ROBOT SALVAJE' 'EL CONTADOR 2' 'STAR WARS EPI.3 LA VENGAZA DE LOS SIHT'
 'PUENTE EN LLAMAS' 'UNTIL DAWN : NOCHE DE TERROR'
 'Star Wars: Episodio III - La venganza de los Sith'
 'Star Wars: Episodio III - La venganza de los sith' 'Rey de reyes'
 'El rey de

In [157]:
import re
from fuzzywuzzy import fuzz

def normalize(text):
    """
    Normalize the input string by converting to lowercase, 
    removing extra whitespaces, and stripping punctuation.
    """
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.strip()

def unify_movie_names(df, column_name='Nombre', similarity_threshold=55, new_col_name='Nombre_normalizado'):
    """
    Unify movie names in a DataFrame by creating a mapping of movie titles that are similar.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame containing movie names.
        column_name (str): The name of the column with the movie names to be normalized.
        similarity_threshold (int): The similarity threshold (0-100) used by fuzzywuzzy's ratio to classify
                                    two titles as equivalent. Default is 80.
        new_col_name (str): The name of the new column that will store the unified movie names.
    
    Returns:
        pd.DataFrame: The DataFrame with an additional column of unified movie names.
        dict: A mapping dictionary where each original movie title is mapped to its canonical title.
    """
    mapping = {}
    canonical_titles = []
    
    # Retrieve the unique titles in the given column
    unique_titles = df[column_name].unique()
    
    # Process each unique title
    for title in unique_titles:
        norm_title = normalize(title)
        matched = False
        # Compare with already-established canonical titles
        for canonical in canonical_titles:
            if fuzz.token_set_ratio(norm_title, normalize(canonical)) >= similarity_threshold: #len(set(norm_title.split()) & set(normalize(canonical).split())) >= 2: #
                mapping[title] = canonical
                matched = True
                break
        if not matched:
            # If no similar title is found, this title becomes a new canonical title
            canonical_titles.append(title)
            mapping[title] = title
    
    # Map the canonical titles to the new column in the DataFrame
    df[new_col_name] = df[column_name].map(mapping)
    
    return df, mapping

    
   

In [158]:
# Apply the function
pelis, mapping = unify_movie_names(pelis)

print("Updated DataFrame:")
print(pelis)

print("\nMapping Dictionary:")
print(mapping)


Updated DataFrame:
                                Nombre Horario FormatoImagen FormatoIdioma  \
Fecha                                                                        
2025-04-23                   MINECRAFT   18:30            2D       Español   
2025-04-23                   PECADORES   21:00            2D       Español   
2025-04-23                   MAZEL TOV   21:15            2D       Español   
2025-04-23                   MAZEL TOV   19:00            2D       Español   
2025-04-23                BLANCANIEVES   19:40            2D       Español   
...                                ...     ...           ...           ...   
2025-05-07  UNTIL DAWN NOCHE DE TERROR   14:00            2D       Español   
2025-05-07  UNTIL DAWN NOCHE DE TERROR   16:50            2D       Español   
2025-05-07  UNTIL DAWN NOCHE DE TERROR   19:40            2D       Español   
2025-05-07           Björk: Cornucopia   19:50            2D   Subtitulado   
2025-05-07               Thunderbolts*   22:3

In [159]:
sorted_mapping = dict(sorted(mapping.items()))
sorted_mapping

{'BJÖRK CORNUCOPIA': 'BJÖRK CORNUCOPIA',
 'BLANCA NIEVES': 'BLANCANIEVES',
 'BLANCANIEVES': 'BLANCANIEVES',
 'Björk: Cornucopia': 'BJÖRK CORNUCOPIA',
 'Blanca Nieves': 'BLANCANIEVES',
 'Cónclave': 'Cónclave',
 'DROP AMENAZA ANONIMA': 'DROP AMENAZA ANONIMA',
 'DROP: Amenaza Anónima': 'DROP AMENAZA ANONIMA',
 'DROP: Amenaza anónima': 'DROP AMENAZA ANONIMA',
 'EL AMATEUR OPERACION VENGANZA': 'El Amateur: Operación Venganza',
 'EL AMATEUR: OPERACION VENGANZA': 'El Amateur: Operación Venganza',
 'EL CONTADOR 2': 'PECADORES',
 'EL REY DE REYES': 'EL REY DE REYES',
 'El Amateur: Operación Venganza': 'El Amateur: Operación Venganza',
 'El Contador 2': 'PECADORES',
 'El contador 2': 'PECADORES',
 'El día que la Tierra explotó: Una película de Looney Tunes': 'Looney Tunes',
 'El rey de reyes': 'EL REY DE REYES',
 'Flow': 'Flow',
 'LA ZURDA': 'LA ZURDA',
 'La venganza de los Sith 20 Aniversario': 'La venganza de los Sith 20 Aniversario',
 'Looney Tunes': 'Looney Tunes',
 'Looney Tunes: El Día que

In [160]:
pelis.head(50)

,Nombre,Horario,FormatoImagen,FormatoIdioma,Cine,Nombre_normalizado
Fecha,,,,,,
2025-04-23,MINECRAFT,18:30,2D,Español,Monumental,MINECRAFT
2025-04-23,PECADORES,21:00,2D,Español,Monumental,PECADORES
2025-04-23,MAZEL TOV,21:15,2D,Español,Monumental,MAZEL TOV
2025-04-23,MAZEL TOV,19:00,2D,Español,Monumental,MAZEL TOV
2025-04-23,BLANCANIEVES,19:40,2D,Español,Monumental,BLANCANIEVES
2025-04-23,Mazel Tov,19:40,2D,Español,Las Tipas,MAZEL TOV
2025-04-23,Mazel Tov,20:00,2D,Español,Las Tipas,MAZEL TOV
2025-04-23,Mazel Tov,21:00,2D,Español,Las Tipas,MAZEL TOV
2025-04-23,Mazel Tov,22:10,2D,Español,Las Tipas,MAZEL TOV


In [161]:
pelis.drop(columns=['Nombre'], inplace=True)
pelis.rename(columns={'Nombre_normalizado': 'Nombre'}, inplace=True)

---

In [162]:
def convert_caps_to_title(text):
    """
    Convert any word in all CAPS to a version with only the first letter capitalized.
    
    Parameters:
        text (str): The input string to process.
        
    Returns:
        str: The processed string with all-CAPS words converted.
    """
    def replacer(match):
        word = match.group(0)
        # Check if the entire word is uppercase (this works with Unicode letters as well)
        if word.isupper():
            return word.capitalize()
        return word  # Return unchanged if not fully uppercase
    
    # This regex will match any word composed of word characters.
    # The re.UNICODE flag ensures Unicode characters are considered word characters.
    return re.sub(r'\b\w+\b', replacer, text, flags=re.UNICODE)


def normalize_formato_idioma(df, column_name='FormatoIdioma'):
    """
    Normalizes the words in CAPs within the given column of the DataFrame so that 
    they use only the first letter in caps.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing the column to process.
        column_name (str): The name of the column to apply the normalization on.
    
    Returns:
        pd.DataFrame: The DataFrame with the updated column.
    """
    # Ensure we work with string data by converting the column values
    df[column_name] = df[column_name].astype(str).apply(convert_caps_to_title)
    return df


In [163]:
# Apply the normalization function on the 'FormatoIdioma' column
pelis = normalize_formato_idioma(pelis, column_name='FormatoIdioma')

In [164]:
pelis.loc[pelis['FormatoIdioma'] != 'Subtitulado', 'FormatoIdioma'] = 'Castellano'

In [165]:
pelis.head(50)

,Horario,FormatoImagen,FormatoIdioma,Cine,Nombre
Fecha,,,,,
2025-04-23,18:30,2D,Castellano,Monumental,MINECRAFT
2025-04-23,21:00,2D,Castellano,Monumental,PECADORES
2025-04-23,21:15,2D,Castellano,Monumental,MAZEL TOV
2025-04-23,19:00,2D,Castellano,Monumental,MAZEL TOV
2025-04-23,19:40,2D,Castellano,Monumental,BLANCANIEVES
2025-04-23,19:40,2D,Castellano,Las Tipas,MAZEL TOV
2025-04-23,20:00,2D,Castellano,Las Tipas,MAZEL TOV
2025-04-23,21:00,2D,Castellano,Las Tipas,MAZEL TOV
2025-04-23,22:10,2D,Castellano,Las Tipas,MAZEL TOV


---

# fin

In [166]:
print(pelis['Nombre'].unique())

['MINECRAFT' 'PECADORES' 'MAZEL TOV' 'BLANCANIEVES' 'DROP AMENAZA ANONIMA'
 'El Amateur: Operación Venganza' 'Tierras perdidas' 'Rescate implacable'
 'The Chosen: La última cena' 'Looney Tunes' 'Until dawn: noche de terror'
 'Flow' 'Cónclave' 'La venganza de los Sith 20 Aniversario'
 'Puente en llamas' 'Pink Floyd at Pompeii – MCMLXXII' 'EL REY DE REYES'
 'ROBOT SALVAJE' 'Thunderbolts*' 'BJÖRK CORNUCOPIA' 'LA ZURDA']


In [167]:
pelis.to_csv('pelis_combinado_clean.csv', index=True)